# Simple Item-Based Collaborative Filtering

This notebook demonstrates a simple implementation of item-based collaborative filtering using the MovieLens dataset.

The process includes:
1. Loading the MovieLens dataset
2. Training an item-based KNN model
3. Finding similar items to those rated by a test user
4. Generating personalized recommendations

In [1]:
# Import required libraries
from recsys.MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter

### Setup and Data Loading

We'll use user ID 85 as our test subject and set k=10 for the number of similar items to consider.

In [ ]:
# Set parameters
test_subject = '85'
k = 10

# Load the MovieLens dataset
lens, data, ratings = MovieLens.load()

# Build the training set
train_set = data.build_full_trainset()

### Train the Model

We'll train an item-based KNN model using cosine similarity.

In [ ]:
# Configure and train the model
sim_options = {
    'name': 'cosine',
    'user_based': False
}

model = KNNBasic(sim_options=sim_options)
model.fit(train_set)
similarity_matrix = model.compute_similarities()

### Generate Recommendations

For our test user, we'll:
1. Get their top k rated items
2. Find similar items to those they rated highly
3. Generate recommendations based on item similarity and ratings

In [ ]:
# Get the test user's inner ID
testUserInnerID = train_set.to_inner_uid(test_subject)

# Get the top K items rated by the user
testUserRatings = train_set.ur[testUserInnerID]
k_neighbours = heapq.nlargest(k, testUserRatings, key=lambda t: t[1])

# Find similar items to those rated highly by the user
candidates = defaultdict(float)
for item_id, rating in k_neighbours:
    similarityRow = similarity_matrix[item_id]
    for inner_id, score in enumerate(similarityRow):
        candidates[inner_id] += score * (rating / 5.0)

# Track items the user has already seen
watched = {}
for item_id, rating in train_set.ur[testUserInnerID]:
    watched[item_id] = 1

# Get and display top recommendations
print("Top 10 recommendations for user", test_subject, ":\n")
pos = 0
for item_id, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not item_id in watched:
        movie_id = train_set.to_raw_iid(item_id)
        print(lens.get_movie_name(int(movie_id)), rating_sum)
        pos += 1
        if (pos > 10):
            break